# Hidden Markov Models for EEG Classification

We propose using a Hidden Markov Model (HMM) for classification of electroencephalograms (EEGs). We are solving a multi-class classification problem. Due to the inherent time-series structure of EEG data, and the noisiness of the single-channel EEG, we thought it could be appropriate to model this with an HMM, where the observed states are the amplitudes of the EEG signal. A similar approach was previously used in Längkvist et. al. (2012) with EEG data to identify stages of sleep.

Our dataset is drawn from the [UCI Epileptic Seizure Recognition Dataset](https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition). 

The classes are as follows:

5 - eyes open

4 - eyes closed

3 - EEG activity from the healthy brain area (in a patient with a tumor)

2 - EEG from the area where the tumor was located

1 - Recording of seizure activity

In [46]:
from hmmlearn import hmm
import util
import pandas as pd
import numpy as np

%%matplotlib inline

UsageError: Line magic function `%%matplotlib` not found.


## Load data

Now we load the data using pandas. The leftmost column is the ID; the columns prefixed with "X" are the EEG amplitudes over one second (178Hz sample rate), and the rightmost column "y" is the class label. Note that $y \in \{1, 2, 3, 4, 5\}$.

In [8]:
data_path = "./data.csv"

df = pd.read_csv(data_path)
df.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


Now, we break group dataframe by class label:

In [18]:
df1 = df[df['y'] == 1]
df2 = df[df['y'] == 2]
df3 = df[df['y'] == 3]
df4 = df[df['y'] == 4]
df5 = df[df['y'] == 5]
df_list = [df1, df2, df3, df4, df5]

To create a balanced test set, we do a 70-30 split of each class, and use the 30% to create an overall test set.

In [66]:
np.random.seed(42) # for reproducibility

train_dfs = []
test_dfs = []
for sub_df in df_list:
    mask = np.random.rand(len(sub_df)) < 0.7
    
    train = sub_df[mask]
    train_dfs.append(train)
    
    test = sub_df[~mask]
    test_dfs.append(test)

In [67]:
print(test_df.head())
for i, df_ in enumerate(train_dfs):
    print("DataFrame #{}".format(i))
    print(df_.head())

    Unnamed: 0   X1   X2   X3   X4   X5    X6   X7   X8   X9  ...  X170  X171  \
8   X19.V1.874 -278 -246 -215 -191 -177  -167 -157 -139 -118  ...  -400  -379   
11  X21.V1.724 -167 -230 -280 -315 -338  -369 -405 -392 -298  ...   423   434   
27   X3.V1.744 -340 -381 -376 -336 -275  -204 -131  -70  -16  ...   114   -39   
42  X18.V1.916  512  351  -90 -534 -944 -1002 -795 -292  286  ...  -747    16   
58  X21.V1.134  143  134  124  112   94    65   25  -12  -48  ...    12    35   

    X172  X173  X174  X175  X176  X177  X178  y  
8   -336  -281  -226  -174  -125   -79   -40  1  
11   416   374   319   268   215   165   103  1  
27  -185  -293  -351  -379  -380  -350  -308  1  
42   422   622   610   371   -99  -652 -1089  1  
58    43    45    27     9   -14   -38   -63  1  

[5 rows x 180 columns]
DataFrame #0
    Unnamed: 0   X1   X2   X3   X4   X5   X6   X7   X8   X9  ...  X170  X171  \
1   X15.V1.924  386  382  356  331  320  315  307  272  244  ...   164   150   
20  X23.V1.964  

## Experimental Methods

In our usage of an HMM, we make two key assumptions of the Markov property and independence. That is: 

For a HMM with hidden states $\{h^{(0)}, h^{(1)}, ... , h^{(t)}\}$ and observed states $\{x^{(0)}, x^{(1)}, ... x^{(t)}\}$, $h^{(i)}$ is dependent only on $h^{(i-1)}$. Similarly, the independence assumption tells us that $x^{(i)}$ is dependent only on $h^{(i)}$. Most notably, this implies $x^{(i)} \perp \!\!\! \perp x^{(i-1)}$.

We also assume a univariate Gaussian emission prior; that is, $x^{(i)} \sim \mathcal{N}(\mu^{(i)}_j, \sigma^{(i)}_j)$ for each class $j$ at time step $i$.

We use a sequence classification approach, meaning that we train 5 HMMs, and output a vector representing the log-likelilhood that each HMM could have generated a given sequence. This corresponds directly to distribution parameter estimation in generative models. To quantify the error with respect to other models (e.g. vanilla softmax regression), we can calculate the categorical cross-entropy of our predictions at test time, in which we represent the true class labels as one-hot vectors.

For our methods, we use the hmmlearn library.

In [ ]:
n = 1200 # training size - max is len(df). 
n_h = 12 # tune this!
verbose = True # for viewing the training log in real time - useful for sanity-checking

hmms = []
for i, df in enumerate(train_dfs):
    hmm_curr = hmm.GaussianHMM(n_components=n_h, covariance_type="spherical", 
                               n_iter=100, random_state=42, verbose=verbose)
    x_curr = np.array(df[df.columns[1:-1]])[:n, :] # shape := (n_examples, timesteps)
    lengths = [x_curr.shape[1]] * x_curr.shape[0]
    assert len(lengths) == x_curr.shape[0]
    
    print("Fitting HMM #{}".format(i+1))
    hmm_curr.fit(x_curr.reshape(-1, 1), lengths)
    hmms.append(hmm_curr)

Fitting HMM #1
         1    -1627416.1655             +nan
         2    -1496286.0481     +131130.1173
         3    -1457146.9477      +39139.1004
         4    -1421879.9076      +35267.0401
         5    -1389567.4602      +32312.4474
         6    -1362771.0272      +26796.4330
         7    -1343120.0628      +19650.9644


## Evaluation

First, we see how our fitted HMMs perform on our training and dev sets on the classification task using accuracy. We will also compute per-class accuracy.

In [114]:
def evaluate(hmms, x, y):
    preds = np.zeros((x.shape[0],))
    for i in range(x.shape[0]):
        scores = np.zeros((5,))
        for j, hmm in enumerate(hmms):
            curr_slice = x.iloc[i, :].to_numpy()
            scores[j] = hmm.score(curr_slice.reshape(1, -1))
        preds[i] = np.argmax(scores) + 1
    acc = np.count_nonzero(np.array(preds) == y) / y.size
    return acc

In [115]:
accs = []
for df in train_dfs:
    x = df[df.columns[1:-1]].iloc[:n, :]
    assert x.shape[1] == 178
    y = df[df.columns[-1]].iloc[:n]
    accs.append(evaluate(hmms, x, y))
print(accs)

ValueError: shapes (1,178) and (1,5) not aligned: 178 (dim 1) != 1 (dim 0)